## Imports

In [1]:
import gym
import argparse

import torch
import torch.optim as optim

from tensorboardX import SummaryWriter

from src import actions, agents, runner, common, wrapper, runner
from src.models import dqn_model
from src.common import hyperparameters, logger
from src.memory import ExperienceReplayBuffer

## Parameters

In [2]:
#CONFIG
params = hyperparameters.PARAMS['pong']
# parser = argparse.ArgumentParser()
# parser.add_argument("--cuda", default=False, action="store_true", help="Enable Cuda")
# args = parser.parse_args()
device = torch.device("cpu") #cuda or cpu

## Environment

In [3]:
#INIT ENV
env = gym.make(params['env_name'])
env = wrapper.wrap_dqn(env)


## Logging

In [4]:
#LOGGING
writer = SummaryWriter(comment="-" + params['run_name'] + "-basic")

## Network

In [5]:
#NETWORK
net = dqn_model.DQN(env.observation_space.shape, env.action_space.n).to(device)
tgt_net = agents.TargetNetwork(net)


## Agent

In [6]:
#AGENT
selector = actions.EpsilonGreedyActionSelector(epsilon=params['epsilon_start'])
epsilon_tracker = logger.EpsilonTracker(selector, params)
agent = agents.DQNAgent(net, selector, device=device)

## Runner

In [7]:
#RUNNER
exp_source = runner.ExperienceSourceFirstLast(env, agent, gamma=params['gamma'],steps_count=1)
buffer = ExperienceReplayBuffer(exp_source,buffer_size=params['replay_size'])
optimizer = optim.Adam(net.parameters(), lr=params['learning_rate'])

frame_idx = 0

## Training 

In [1]:
#TRAIN
with logger.RewardTracker(writer, params['stop_reward']) as reward_tracker:
    while True:
        frame_idx += 1
        buffer.populate(1)
        epsilon_tracker.frame(frame_idx)

        new_rewards = exp_source.pop_total_rewards()
        if new_rewards:
            if reward_tracker.reward(new_rewards[0], frame_idx, selector.epsilon):
                break

        if len(buffer) < params['replay_initial']:
            continue

        #learning step
        optimizer.zero_grad()
        batch = buffer.sample(params['batch_size'])
        loss_v = agent.calc_loss(batch, net, tgt_net.target_model,params['gamma'],device)
        loss_v.backward()
        optimizer.step()

        if frame_idx % params['target_net_sync'] == 0:
            tgt_net.sync()
